In [1]:
import time
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import particles

from scipy.linalg import block_diag
from particles.collectors import Moments
from particles import state_space_models as ssm
from particles import distributions as dists

plt.style.use("ggplot")

In [2]:
# to be removed
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=119
)

In [3]:
nb_timesteps = 700  # observations.shape[0]

true_states = pd.read_csv("data/normal/states.csv")[0:nb_timesteps]
observations = pd.read_csv("data/normal/observations.csv")[0:nb_timesteps]

In [4]:
from MechanicalModel import state_to_obs

In [5]:
class TwoLegModel(ssm.StateSpaceModel):
    def __init__(
        self,
        dt=0.01,
        dim_states=18,
        dim_observations=20,
        femur_left=0.5,
        fibula_left=0.6,
        femur_right=0.5,
        fibula_right=0.6,
        pos_imu0=0.34,
        pos_imu1=0.29,
        pos_imu2=0.315,
        pos_imu3=0.33,
        b0=np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        factor_Q0=0.1,
        lambda_x=10000.0,
        lambda_y=1000.0,
        lambda_phi=10000000.0,
        sigma_imu_acc=0.1,
        sigma_imu_gyro=0.1,
        sigma_press_velo=0.1,
        sigma_press_acc=1.0,
    ):
        ssm.StateSpaceModel().__init__()
        self.dt = dt
        self.dim_states = dim_states
        self.dim_observations = dim_observations
        self.A = np.zeros((dim_states, dim_states))
        self.set_process_transition_matrix()
        self.g = 9.81
        self.len_legs = np.array([femur_left, fibula_left, femur_right, fibula_right])
        self.pos_imus = np.array([pos_imu0, pos_imu1, pos_imu2, pos_imu3])
        self.b0 = b0
        self.factor_Q0 = factor_Q0
        self.Q0 = self.factor_Q0 * np.eye(self.dim_states)
        self.lambda_x = lambda_x
        self.lambda_y = lambda_y
        self.lambda_phi = lambda_phi
        self.sigma_imu_acc = sigma_imu_acc
        self.sigma_imu_gyro = sigma_imu_gyro
        self.sigma_press_velo = sigma_press_velo
        self.sigma_press_acc = sigma_press_acc
        self.Q = np.zeros((self.dim_states, self.dim_states))
        self.set_process_covariance()
        self.V = np.zeros((self.dim_observations, self.dim_observations))
        self.set_observation_covariance()

    def set_process_transition_matrix(self):
        self.A = np.eye(self.dim_states)
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row + 6 == col:
                    self.A[row, col] = self.dt
                if row + 12 == col:
                    self.A[row, col] = self.dt**2 / 2.0

    def set_process_covariance(self):
        block_size = self.dim_states // 3
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row < block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**5 / 20.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**4 / 8.0
                        self.Q[col, row] = self.dt**4 / 8.0
                    elif row + 12 == col:
                        self.Q[row, col] = self.dt**3 / 6.0
                        self.Q[col, row] = self.dt**3 / 6.0
                elif block_size <= row < 2 * block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**3 / 3.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**2 / 2.0
                        self.Q[col, row] = self.dt**2 / 2.0
                elif 2 * block_size <= row:
                    if row == col:
                        self.Q[row, col] = self.dt
        idx_groups = [[0, 6, 12], [1, 7, 13], [2, 8, 14], [3, 9, 15], [4, 10, 16], [5, 11, 17]]
        scale_factors = [
            self.lambda_x,
            self.lambda_y,
            self.lambda_phi,
            self.lambda_phi,
            self.lambda_phi,
            self.lambda_phi,
        ]
        for factor, idxs in zip(scale_factors, idx_groups):
            for row, col in itertools.product(idxs, idxs):
                self.Q[row, col] *= factor

    def set_observation_covariance(self):
        self.V = np.diag(
            [
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
            ]
        )

    def state_transition(self, xp):
        return np.matmul(self.A, xp.T).T

    def state_to_observation(self, x):
        return state_to_obs(x, self.len_legs, self.pos_imus)

    def PX0(self):
        return dists.MvNormal(loc=self.b0, cov=self.Q0)

    def PX(self, t, xp):
        return dists.MvNormal(loc=self.state_transition(xp), cov=self.Q)

    def PY(self, t, xp, x):
        return dists.MvNormal(loc=self.state_to_observation(x), cov=self.V)

In [6]:
def define_model(lx, ly, lphi, simu_a, simu_g, spress_v, spress_a):
    model = TwoLegModel(
        lambda_x=lx,
        lambda_y=ly,
        lambda_phi=lphi,
        sigma_imu_acc=simu_a,
        sigma_imu_gyro=simu_g,
        sigma_press_velo=spress_v,
        sigma_press_acc=spress_a,
    )
    return model

In [7]:
def run_particle_filter(ssm_model, obs, model_parameters, nb_particles=1000, ESSrmin=0.5):
    fk_model = ssm.Bootstrap(ssm=ssm_model, data=obs.to_numpy())
    pf = particles.SMC(
        fk=fk_model,
        N=nb_particles,
        ESSrmin=ESSrmin,
        store_history=True,
        collect=[Moments()],
    )
    pf.run()
    loglikelihood = pf.summaries.logLts[-1]
    print("-------------------------------------------------------------")
    print("_".join(f"{key}={value}" for key, value in parameters.items()))
    print(f"Log likelihood = {loglikelihood:.3E}")
    print(f"Resampled {np.sum(pf.summaries.rs_flags)} of totally {nb_timesteps} steps.")
    print("-------------------------------------------------------------")
    return pf

In [8]:
def export_plots(true_states, pf, parameters):
    particles_mean = np.array([p["mean"] for p in pf.summaries.moments])
    particles_var = np.sqrt([p["var"] for p in pf.summaries.moments])
    title = "True states vs filtered particles"
    folder_name = "_".join(f"{key}={value}" for key, value in parameters.items())
    if not os.path.exists(f"results/{folder_name}"):
        os.mkdir(f"results/{folder_name}")
    for i, state_name in enumerate(true_states.columns):
        fig = plt.figure(figsize=(10, 6))
        plt.plot(true_states[state_name], label="True states")
        plt.plot(particles_mean[:, i], color="blue", label="Filtered mean with std")
        plt.fill_between(
            true_states.index,
            particles_mean[:, i] - particles_var[:, i],
            particles_mean[:, i] + particles_var[:, i],
            color="blue",
            alpha=0.3,
        )
        plt.legend()
        plt.title(r"State " + state_name)
        plt.savefig(f"results/{folder_name}/{state_name}.pdf")

        fig = plt.figure(figsize=(10, 6))
        plt.plot(pf.summaries.ESSs, label="ESS")
        window_avg = np.ones(10) / 10.0
        moving_avg = np.convolve(pf.summaries.ESSs, window_avg, "same")
        plt.plot(moving_avg, label="moving average")
        plt.legend()
        plt.title(f"Essential sample size for N={pf.N}")
        plt.savefig(f"results/{folder_name}/ESS.pdf")

In [ ]:
lambda_x_arr = [1.0]
lambda_y_arr = [0.1]
lambda_phi_arr = [10000.0]
sigma_imu_acc_arr = [10.0, 100.0, 1000.0]
sigma_imu_gyro_arr = [10.0, 100.0, 1000.0]
sigma_press_velo_arr = [10.0, 100.0, 1000.0]
sigma_press_acc_arr = [100.0, 1000.0, 10000.0]
for lx, ly, lphi, sia, sig, spv, spa in itertools.product(
    lambda_x_arr,
    lambda_y_arr,
    lambda_phi_arr,
    sigma_imu_acc_arr,
    sigma_imu_gyro_arr,
    sigma_press_velo_arr,
    sigma_press_acc_arr,
):
    parameters = {
        "lambda_x": lx,
        "lambda_y": ly,
        "lambda_phi": lphi,
        "sigma_imu_acc": sia,
        "sigma_imu_gyro": sig,
        "sigma_press_velo": spv,
        "sigma_press_acc": spa,
    }
    model = define_model(lx, ly, lphi, sia, sig, spv, spa)
    pf = run_particle_filter(model, observations, parameters)
    export_plots(true_states, pf, parameters)

-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=100.0
Log likelihood = -3.635E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------


/tmp/ipykernel_74956/526582806.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10, 6))


-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=1000.0
Log likelihood = -3.788E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=10000.0
Log likelihood = -4.270E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=10.0_sigma_press_velo=100.0_sigma_press_acc=100.0
Log likelihood = -4.108E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
------------------------------------------------

/tmp/ipykernel_74956/526582806.py:3: RuntimeWarning: invalid value encountered in sqrt
  particles_var = np.sqrt([p["var"] for p in pf.summaries.moments])


-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=1000.0_sigma_press_velo=100.0_sigma_press_acc=100.0
Log likelihood = -5.042E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=1000.0_sigma_press_velo=100.0_sigma_press_acc=1000.0
Log likelihood = -5.085E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------


...

/tmp/ipykernel_74956/526582806.py:3: RuntimeWarning: invalid value encountered in sqrt
  particles_var = np.sqrt([p["var"] for p in pf.summaries.moments])


-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=1000.0_sigma_press_velo=1000.0_sigma_press_acc=100.0
Log likelihood = -4.977E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=1000.0_sigma_press_velo=1000.0_sigma_press_acc=1000.0
Log likelihood = -6.170E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------


/tmp/ipykernel_74956/526582806.py:3: RuntimeWarning: invalid value encountered in sqrt
  particles_var = np.sqrt([p["var"] for p in pf.summaries.moments])


-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=10.0_sigma_imu_gyro=1000.0_sigma_press_velo=1000.0_sigma_press_acc=10000.0
Log likelihood = -5.386E+04
Resampled 699 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=100.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=100.0
Log likelihood = -4.001E+04
Resampled 698 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=100.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=1000.0
Log likelihood = -4.342E+04
Resampled 349 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------

-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=100.0_sigma_imu_gyro=1000.0_sigma_press_velo=1000.0_sigma_press_acc=1000.0
Log likelihood = -5.568E+04
Resampled 349 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=100.0_sigma_imu_gyro=1000.0_sigma_press_velo=1000.0_sigma_press_acc=10000.0
Log likelihood = -5.889E+04
Resampled 349 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------------------------------
lambda_x=1.0_lambda_y=0.1_lambda_phi=10000.0_sigma_imu_acc=1000.0_sigma_imu_gyro=10.0_sigma_press_velo=10.0_sigma_press_acc=100.0
Log likelihood = -4.620E+04
Resampled 383 of totally 700 steps.
-------------------------------------------------------------
-------------------------------------